**Important:** Be sure to use the specified names for the trigger functions and triggers. 

- fn_update_gpa
- tr_update_gpa

- assign_grade

- fn_update_deans_list
- tr_update_deans_list

Grading will assume the specified names have been used.

## Assignment 4 /  Triggers
### 95-568 Data Management and Security, Summer 2019

In [3]:
%load_ext sql
%sql postgresql://isdb@localhost/postgres

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: isdb@postgres'

## Question 1: Trigger for Calculating GPA

In [4]:
%%sql

drop table if exists Students CASCADE;
drop table if exists Courses CASCADE;
drop table if exists Grades CASCADE;

create table Students(
    primary key (sId),
    sId int,
    name text,
    gpa numeric(3,2) default 0.0
);

create table Courses(
    primary key (cId),
    cId int,  
    name text,
    units int
);

create table Grades(
    primary key(sId, cId),
    sId int references Students(sId),
    cId int references Courses(cId),  
    grade numeric(2,1) 
);

 
INSERT INTO Students (sid, name) -- <-- gpa not entered
     VALUES (1, 'Jack'),
            (2, 'Jill'),
            (3, 'Pat');
            
INSERT INTO Courses (cId, name, units)
     VALUES (1, 'Database Design and Development', 12),
            (2, 'Practical Data Science', 9),
            (3, 'Big Data Analytics', 6),
            (4, 'Machine Learning', 9),
            (5, 'Decision Support Systems', 9),
            (6, 'Business applications of AI', 9)
            ;
            
INSERT INTO Grades( sId, cId, grade)
     VALUES (1, 1, 4.0),
            (1, 2, 3.3),
            (2, 1, 3.7);
            

 * postgresql://isdb@localhost/postgres
Done.
Done.
Done.
Done.
Done.
Done.
3 rows affected.
6 rows affected.
3 rows affected.


[]

#### A handy python function that prints all three tables at the same time

In [5]:
# Note the use of %sql to capture outuput of an SQL statement as a Python list

def print_all_tables():
    s = %sql select * from Students; 
    c = %sql select * from Courses;
    g = %sql select * from Grades;
    print("\nStudents:\n%s" % s)
    print("\nCourses:\n%s" % c)
    print("\nGrades:\n%s" % g)

In [6]:
print_all_tables()

 * postgresql://isdb@localhost/postgres
3 rows affected.
 * postgresql://isdb@localhost/postgres
6 rows affected.
 * postgresql://isdb@localhost/postgres
3 rows affected.

Students:
+-----+------+------+
| sid | name | gpa  |
+-----+------+------+
|  1  | Jack | 0.00 |
|  2  | Jill | 0.00 |
|  3  | Pat  | 0.00 |
+-----+------+------+

Courses:
+-----+---------------------------------+-------+
| cid |               name              | units |
+-----+---------------------------------+-------+
|  1  | Database Design and Development |   12  |
|  2  |      Practical Data Science     |   9   |
|  3  |        Big Data Analytics       |   6   |
|  4  |         Machine Learning        |   9   |
|  5  |     Decision Support Systems    |   9   |
|  6  |   Business applications of AI   |   9   |
+-----+---------------------------------+-------+

Grades:
+-----+-----+-------+
| sid | cid | grade |
+-----+-----+-------+
|  1  |  1  |  4.0  |
|  1  |  2  |  3.3  |
|  2  |  1  |  3.7  |
+-----+-----+

## TODO 1: Write a trigger function `fn_update_gpa` that will recalculate the GPA of a student when ever a new row is inserted into the Grades table i.e., a student is assigned a grade for a course


SELECT (SELECT sum(G.grade * C.units)
                    FROM Students AS S 
                    JOIN Grades AS G 
                      ON S.sid=G.sid
                    JOIN Courses AS C
                      ON G.cid=C.cid
                    WHERE S.sid=new.sid)/
                    
                    (SELECT sum(C.units)
                       FROM Students AS S 
                    JOIN Grades AS G 
                      ON S.sid=G.sid
                    JOIN Courses AS C
                      ON G.cid=C.cid
                    WHERE S.sid=new.sid)

In [7]:
%%sql

DROP FUNCTION IF EXISTS fn_update_gpa(); 

CREATE FUNCTION fn_update_gpa() RETURNS trigger 
LANGUAGE plpgsql AS
$$
BEGIN 
    UPDATE Students 
       SET gpa = (SELECT sum(G.grade * C.units) / sum(C.units) test
                    FROM Students AS S 
                    JOIN Grades AS G 
                      ON S.sid=G.sid
                    JOIN Courses AS C
                      ON G.cid=C.cid
                   WHERE S.sid = new.sid)
                    
     WHERE sid = new.sid;
    RETURN NULL;
END 
$$;

 * postgresql://isdb@localhost/postgres
Done.
Done.


[]

## TODO 2: Write the Trigger `tr_update_gpa`
Use the event:
AFTER INSERT OR UPDATE 

In [8]:
%%sql

DROP TRIGGER IF EXISTS tr_update_gpa ON Grades;

CREATE TRIGGER tr_update_gpa 
AFTER INSERT ON Grades
FOR EACH ROW 
    EXECUTE PROCEDURE fn_update_gpa();

 * postgresql://isdb@localhost/postgres
Done.
Done.


[]

## TODO 3: Complete the function `assign_grade` 

`assign_grade(2, 3, 4.0)` creates an abstraction around the process of assigning a grade to a student.

The grades table has rows of `(sid, cid, grade)`.  Your function needs to determine if the `sid` `cid` combination already exists in the table, and if so it updates an existing grade.  Otherwise it will insert the new grade.

Use the boolean functino `EXISTS(SELECT _____ )` to determine if a row exists or not.

In [9]:
%%sql

DROP FUNCTION IF EXISTS assign_grade(int, int, numeric(2,1));

create function assign_grade(p_sid int, p_cid int, p_grade numeric(2,1)) 
returns void
language plpgsql as
$$
begin
    IF (EXISTS (SELECT G.sid, G.cid 
                  FROM Grades AS G
                 WHERE G.sid=p_sid 
                   AND G.cid=p_cid))
  THEN UPDATE Grades
          SET grade=p_grade
        WHERE sid=p_sid
          AND cid=p_cid;
  ELSE INSERT INTO Grades(sid, cid, grade)
          VALUES (p_sid, p_cid, p_grade);
END IF;
end
$$;

 * postgresql://isdb@localhost/postgres
Done.
Done.


[]

In [10]:
%%sql 

-- Assign Jill((sid=2) the grade of 3.0 for course cid=3
select assign_grade(2, 3, 3.0)

 * postgresql://isdb@localhost/postgres
1 rows affected.


assign_grade
""


### You should now see the GPA calculated for Jill

In [11]:
print_all_tables()

 * postgresql://isdb@localhost/postgres
3 rows affected.
 * postgresql://isdb@localhost/postgres
6 rows affected.
 * postgresql://isdb@localhost/postgres
4 rows affected.

Students:
+-----+------+------+
| sid | name | gpa  |
+-----+------+------+
|  1  | Jack | 0.00 |
|  3  | Pat  | 0.00 |
|  2  | Jill | 3.47 |
+-----+------+------+

Courses:
+-----+---------------------------------+-------+
| cid |               name              | units |
+-----+---------------------------------+-------+
|  1  | Database Design and Development |   12  |
|  2  |      Practical Data Science     |   9   |
|  3  |        Big Data Analytics       |   6   |
|  4  |         Machine Learning        |   9   |
|  5  |     Decision Support Systems    |   9   |
|  6  |   Business applications of AI   |   9   |
+-----+---------------------------------+-------+

Grades:
+-----+-----+-------+
| sid | cid | grade |
+-----+-----+-------+
|  1  |  1  |  4.0  |
|  1  |  2  |  3.3  |
|  2  |  1  |  3.7  |
|  2  |  3  |

In [10]:
# Calculating by hand we see
# cid 3 / 6 units / 3.0
# cid 1 / 12 units / 3.7
%sql select ((6*3.0 + 12*3.7) / (6.0+12.0)) as "gpa";

 * postgresql://isdb@localhost/postgres
1 rows affected.


gpa
3.4666666666666667


##  Question 2:  Write a trigger such that if a person's GPA is >= 3.5 then they are inserted into a Dean's honors table

In [11]:
%%sql

drop table if exists Deans_List;

create table Deans_List (
     sid   int,
     name  text,
     gpa   numeric(3,2)
);

 * postgresql://isdb@localhost/postgres
Done.
Done.


[]

### TODO 4 : Write the `fn_update_deans_list` trigger function

Insert into the Dean's List table only if the gpa >= 3.5

In [12]:
%%sql

DROP FUNCTION IF EXISTS fn_update_deans_list() cascade;

-- To update a students inclusion in the Dean's list you will find
-- it easier to (i) drop the student first (whether they 
-- are included already or not) and (ii) insert them if their GPA is
-- >= 3.5

CREATE FUNCTION fn_update_deans_list() RETURNS trigger 
LANGUAGE plpgsql AS 
$$
BEGIN 
   DELETE FROM Deans_List
    WHERE sid=old.sid
      AND name=old.name
      AND new.gpa < 3.5;
       IF (new.gpa >= 3.5) 
     THEN INSERT INTO Deans_List (sid, name, gpa)
              VALUES (old.sid, old.name, new.gpa);
      END IF;
   return null;    
END $$ ;

 * postgresql://isdb@localhost/postgres
Done.
Done.


[]

### TODO 5 : Write the trigger `tr_update_deans_list`

In [13]:
%%sql 
DROP TRIGGER IF EXISTS tr_update_deans_list ON Students;

CREATE TRIGGER tr_update_deans_list 
AFTER UPDATE ON Students 
FOR EACH ROW
    EXECUTE PROCEDURE fn_update_deans_list();

 * postgresql://isdb@localhost/postgres
Done.
Done.


[]

In [14]:
%%sql 

-- initially the dean's list is empty
select * from Deans_List;

 * postgresql://isdb@localhost/postgres
0 rows affected.


sid,name,gpa


In [15]:
%%sql
-- Jill gets a 4.0 in cid=2 'Practical Data Science'

select assign_grade(2, 2, 4.0)

 * postgresql://isdb@localhost/postgres
1 rows affected.


assign_grade
""


### Jill's GPA should now be 3.64

In [16]:
%sql select ((6*3.0 + 12*3.7 + 9*4.0) / (6.0+12.0+9.0)) as "gpa";

 * postgresql://isdb@localhost/postgres
1 rows affected.


gpa
3.6444444444444444


### Lets check the tables

In [17]:
print_all_tables()

 * postgresql://isdb@localhost/postgres
3 rows affected.
 * postgresql://isdb@localhost/postgres
6 rows affected.
 * postgresql://isdb@localhost/postgres
5 rows affected.

Students:
+-----+------+------+
| sid | name | gpa  |
+-----+------+------+
|  1  | Jack | 0.00 |
|  3  | Pat  | 0.00 |
|  2  | Jill | 3.64 |
+-----+------+------+

Courses:
+-----+---------------------------------+-------+
| cid |               name              | units |
+-----+---------------------------------+-------+
|  1  | Database Design and Development |   12  |
|  2  |      Practical Data Science     |   9   |
|  3  |        Big Data Analytics       |   6   |
|  4  |         Machine Learning        |   9   |
|  5  |     Decision Support Systems    |   9   |
|  6  |   Business applications of AI   |   9   |
+-----+---------------------------------+-------+

Grades:
+-----+-----+-------+
| sid | cid | grade |
+-----+-----+-------+
|  1  |  1  |  4.0  |
|  1  |  2  |  3.3  |
|  2  |  1  |  3.7  |
|  2  |  3  |

## Jill should now be inserted in the Deans_List table

In [18]:
%sql select * from deans_list;

 * postgresql://isdb@localhost/postgres
1 rows affected.


sid,name,gpa
2,Jill,3.64


### Suppose Jill doesn't do well in cid=4

In [19]:
%sql select assign_grade(2, 4, 2.0)

 * postgresql://isdb@localhost/postgres
1 rows affected.


assign_grade
""


In [20]:
print_all_tables()

 * postgresql://isdb@localhost/postgres
3 rows affected.
 * postgresql://isdb@localhost/postgres
6 rows affected.
 * postgresql://isdb@localhost/postgres
6 rows affected.

Students:
+-----+------+------+
| sid | name | gpa  |
+-----+------+------+
|  1  | Jack | 0.00 |
|  3  | Pat  | 0.00 |
|  2  | Jill | 3.23 |
+-----+------+------+

Courses:
+-----+---------------------------------+-------+
| cid |               name              | units |
+-----+---------------------------------+-------+
|  1  | Database Design and Development |   12  |
|  2  |      Practical Data Science     |   9   |
|  3  |        Big Data Analytics       |   6   |
|  4  |         Machine Learning        |   9   |
|  5  |     Decision Support Systems    |   9   |
|  6  |   Business applications of AI   |   9   |
+-----+---------------------------------+-------+

Grades:
+-----+-----+-------+
| sid | cid | grade |
+-----+-----+-------+
|  1  |  1  |  4.0  |
|  1  |  2  |  3.3  |
|  2  |  1  |  3.7  |
|  2  |  3  |

### Since her GPA has now dropped Jill should no longer be on the Dean's list

In [21]:
%sql select * from deans_list;

 * postgresql://isdb@localhost/postgres
0 rows affected.


sid,name,gpa
